In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')



In [2]:
# 1. 데이터 불러오기
def load_data(filepath):
    """
    CSV 파일에서 심장 질환 데이터를 불러오는 함수
    """
    data = pd.read_csv(filepath)
    print(f"데이터 형태: {data.shape}")
    return data



In [3]:
# 2. 데이터 탐색 및 시각화
def explore_data(data):
    """
    데이터 기본 특성 탐색 및 시각화
    """
    # 기본 정보 확인
    print("데이터 기본 정보:")
    print(data.info())
    
    # 수치형 데이터 통계 요약
    print("\n수치형 데이터 통계 요약:")
    print(data.describe())
    
    # 결측치 확인
    print("\n컬럼별 결측치 수:")
    print(data.isnull().sum())
    
    # 히트맵으로 상관관계 시각화
    plt.figure(figsize=(12, 10))
    numeric_data = data.select_dtypes(include=[np.number])
    sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('심장 질환 데이터 상관관계 히트맵')
    plt.tight_layout()
    plt.savefig('correlation_heatmap.png')
    
    # 심장 질환 사망률 분포 시각화
    plt.figure(figsize=(10, 6))
    sns.histplot(data['Heart Disease Mortality'], kde=True)
    plt.title('심장 질환 사망률 분포')
    plt.xlabel('사망률 (per 100,000)')
    plt.tight_layout()
    plt.savefig('mortality_distribution.png')
    
    # 지역별 심장 질환 사망률 시각화 (상위 15개 지역)
    plt.figure(figsize=(12, 8))
    location_mortality = data.groupby('LocationDesc')['Heart Disease Mortality'].mean().sort_values(ascending=False).head(15)
    sns.barplot(x=location_mortality.values, y=location_mortality.index)
    plt.title('지역별 평균 심장 질환 사망률 (상위 15개)')
    plt.xlabel('평균 사망률 (per 100,000)')
    plt.tight_layout()
    plt.savefig('location_mortality.png')
    
    # 성별에 따른 심장 질환 사망률 비교
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='Sex', y='Heart Disease Mortality', data=data)
    plt.title('성별에 따른 심장 질환 사망률')
    plt.tight_layout()
    plt.savefig('sex_mortality.png')
    
    # 인종별 심장 질환 사망률 비교
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='ethnicity', y='Heart Disease Mortality', data=data)
    plt.title('인종별 심장 질환 사망률')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('ethnicity_mortality.png')
    
    return

# 3. 데이터 전처리
def preprocess_data(data):
    """
    머신러닝 모델에 사용할 수 있도록 데이터 전처리
    """
    # 타겟 변수 설정
    y = data['Heart Disease Mortality']
    
    # 사용할 특성 선택 (예시: 실제 데이터셋에 맞게 수정 필요)
    cat_features = ['LocationAbbr', 'GeographicLevel', 'Sex', 'ethnicity']
    num_features = ['Year', 'Y_lat', 'X_lon']
    
    # 사용하지 않을 컬럼 제외
    # 'LocationDesc', 'Data_Value_Unit', 'Data_Value_Type', 'LocationID', 'Georeference' 등은 제외
    
    # 범주형/수치형 특성에 대한 전처리 파이프라인 구성
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # ColumnTransformer를 사용하여 전처리 파이프라인 구성
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ])
    
    # 특성 매트릭스 구성
    X = data[num_features + cat_features]
    
    # 훈련/테스트 세트 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, preprocessor



In [ ]:
# 4. 모델 학습 및 평가
def train_and_evaluate_models(X_train, X_test, y_train, y_test, preprocessor):
    """
    다양한 머신러닝 모델을 학습하고 평가
    """
    # 모델 정의
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'SVR': SVR()
    }
    
    # 결과 저장을 위한 딕셔너리
    results = {}
    
    # 각 모델 학습 및 평가
    for name, model in models.items():
        print(f"\nTrain {name}")
        
        # 전처리기와 모델을 파이프라인으로 구성
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # 모델 학습
        pipeline.fit(X_train, y_train)
        
        # 예측
        y_pred = pipeline.predict(X_test)
        
        # 평가 지표 계산
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # 결과 저장
        results[name] = {
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Model': pipeline
        }
        
        print(f"{name} performence:")
        print(f"  - RMSE: {rmse:.2f}")
        print(f"  - MAE: {mae:.2f}")
        print(f"  - R² Score: {r2:.4f}")
    
    return results



In [7]:
# 7. 지리적 시각화 (지도에 심장 질환 사망률 표시)
def geographic_visualization(data):
    """
    지도에 심장 질환 사망률 표시
    """
    try:
        import folium
        from folium.plugins import HeatMap
        
        # 좌표와 사망률 데이터 추출
        geo_data = data[['Y_lat', 'X_lon', 'Heart Disease Mortality']].dropna()
        
        # 중심 좌표 계산
        center_lat = geo_data['Y_lat'].mean()
        center_lon = geo_data['X_lon'].mean()
        
        # 맵 생성
        m = folium.Map(location=[center_lat, center_lon], zoom_start=4)
        
        # 히트맵 데이터 생성
        heat_data = [[row['Y_lat'], row['X_lon'], row['Heart Disease Mortality']] 
                     for _, row in geo_data.iterrows()]
        
        # 히트맵 추가
        HeatMap(heat_data, radius=15).add_to(m)
        
        # 맵 저장
        m.save('heart_disease_heatmap.html')
        print("\n지리적 시각화 완료: heart_disease_heatmap.html 파일로 저장됨")
    except ImportError:
        print("\n지리적 시각화를 위해 folium 패키지가 필요합니다. 'pip install folium'으로 설치할 수 있습니다.")



In [8]:
# 8. 예측 결과와 실제값 비교 시각화
def visualize_predictions(y_test, y_pred, model_name):
    """
    예측값과 실제값 비교 시각화
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('실제값')
    plt.ylabel('예측값')
    plt.title(f'{model_name} 모델: 예측값 vs 실제값')
    plt.tight_layout()
    plt.savefig('predictions_vs_actual.png')



In [ ]:

# 데이터 파일 경로
data_path = "heart_disease_mortality_cleaned.csv"  # 실제 파일 경로로 수정 필요

# 1. 데이터 불러오기
try:
    data = load_data(data_path)
except FileNotFoundError:
    print(f"Error: {data_path} is not found.")

# 2. 데이터 탐색 및 시각화
explore_data(data)

# 3. 데이터 전처리
X_train, X_test, y_train, y_test, preprocessor = preprocess_data(data)

# 4. 모델 학습 및 평가
results = train_and_evaluate_models(X_train, X_test, y_train, y_test, preprocessor)

# 5. 최고 성능 모델 선택
best_model_name = max(results, key=lambda k: results[k]['R2'])
print(f"\nBest Model: {best_model_name} (R² = {results[best_model_name]['R2']:.4f})")


데이터 형태: (34430, 13)
데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34430 entries, 0 to 34429
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     34430 non-null  int64  
 1   LocationAbbr             34430 non-null  object 
 2   LocationDesc             34430 non-null  object 
 3   GeographicLevel          34430 non-null  object 
 4   Heart Disease Mortality  34430 non-null  float64
 5   Data_Value_Unit          34430 non-null  object 
 6   Data_Value_Type          34430 non-null  object 
 7   Sex                      34430 non-null  object 
 8   ethnicity                34430 non-null  object 
 9   LocationID               34430 non-null  int64  
 10  Y_lat                    34406 non-null  float64
 11  X_lon                    34406 non-null  float64
 12  Georeference             34406 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usa